In [9]:
# For the control:
import numpy as np

# For the 3D animation:
from IPython.display import HTML
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D # <--- This is important for 3d plotting 
import matplotlib.animation as manimation; manimation.writers.list()

['pillow', 'html']

## Registering the actual joint positions using DH parameters:

In [7]:
def T(d, theta, r, alpha):
    cost = np.cos(theta)
    sint = np.sin(theta)
    cosa = np.cos(alpha)
    sina = np.sin(alpha)
    return np.array([[cost, -sint*cosa,  sint*sina, r*cost], 
                     [sint,  cost*cosa, -cost*sina, r*sint],
                     [   0,       sina,       cosa,      d],
                     [   0,          0,          0,      1]])

def pos(T):
    return np.array([T[0,3], T[1,3], T[2, 3]])

In [10]:
def getCurrentPos(theta_base, theta_shoulder, theta_elbow):
    Shoulder = T(0.089159, theta_base, 0.134, np.pi/2)
    Shoulder[0,3], Shoulder[1,3] = -Shoulder[1,3], Shoulder[0,3]
    
    Elbow    = T(0, -np.pi/2, -0.425, 0)
    # Elbow    = T(0, theta_shoulder, 0.425, 0)
    Elbow    = Shoulder @ Elbow
    
    Elbow2   = T(0.119, 0, 0, 0)
    # Elbow2   = T(0.119, theta_elbow, 0, 0)
    Elbow2   = Elbow @ Elbow2
    
    # Wrist1   = T(0, -np.pi/2, 0.39225, 0)
    Wrist1   = T(0, 0, -0.39225, 0)
    Wrist1   = Elbow2 @ Wrist1
    
    Wrist2   = T(0.09475, -np.pi/2, 0, np.pi/2)
    Wrist2   = Wrist1 @ Wrist2
    
    Wrist3   = T(0.09475, 0, 0, -np.pi/2)
    Wrist3   = Wrist2 @ Wrist3
    
    Tool   = T(0.0815, 0, 0, 0)
    Tool     = Wrist3 @ Tool
    
    ShoulderPos = pos(Shoulder)
    ElbowPos    = pos(Elbow)
    Elbow2Pos   = pos(Elbow2)
    Wrist1Pos   = pos(Wrist1)
    Wrist2Pos   = pos(Wrist2)
    Wrist3Pos   = pos(Wrist3)
    ToolPos     = pos(Tool)
    
    X = np.array([0,              0, ShoulderPos[0], ElbowPos[0], Elbow2Pos[0], Wrist1Pos[0], Wrist2Pos[0], Wrist3Pos[0], ToolPos[0]])
    Y = np.array([0,              0, ShoulderPos[1], ElbowPos[1], Elbow2Pos[1], Wrist1Pos[1], Wrist2Pos[1], Wrist3Pos[1], ToolPos[1]])
    Z = np.array([0, ShoulderPos[2], ShoulderPos[2], ElbowPos[2], Elbow2Pos[2], Wrist1Pos[2], Wrist2Pos[2], Wrist3Pos[2], ToolPos[2]])
    
    return X, Y, Z

self.Base           = RobotJoint(origin=None,          a=0.134,    d=0.089159, alpha=np.pi/2,  num_bytes=8, num_preceding_bytes=252, numeric_type='!d', note="Position (angle) of the base joint")
self.Shoulder       = RobotJoint(origin=self.Base,     a=-0.425,   d=0,        alpha=0,        num_bytes=8, num_preceding_bytes=260, numeric_type='!d', note="Position (angle) of the shoulder joint")
self.Elbow          = RobotJoint(origin=self.Shoulder, a=0,        d=0.119,    alpha=0,        num_bytes=8, num_preceding_bytes=268, numeric_type='!d', note="Position (angle) of the elbow joint")
self.ElbowEnd       = RobotJoint(origin=self.Elbow,    a=-0.39225, d=0,        alpha=0,        num_bytes=8, num_preceding_bytes=268, numeric_type='!d', note="Position (angle) of the elbow joint")
self.Wrist1         = RobotJoint(origin=self.ElbowEnd, a=0,        d=-0.09475, alpha=np.pi/2,  num_bytes=8, num_preceding_bytes=276, numeric_type='!d', note="Position (angle) of the wrist2 joint")
self.Wrist2         = RobotJoint(origin=self.Wrist1,   a=0,        d=0.09475,  alpha=-np.pi/2, num_bytes=8, num_preceding_bytes=284, numeric_type='!d', note="Position (angle) of the wrist3 joint")
self.Wrist3         = RobotJoint(origin=self.Wrist2,   a=0,        d=-0.0815,  alpha=0,        num_bytes=8, num_preceding_bytes=292, numeric_type='!d', note="Position (angle) of the wrist3 joint")



def init():
    pass
    
def rotate_base(i):
    theta_base     = 2*np.pi/17 * i
    theta_shoulder = np.pi/2 + np.sin(np.pi/17 * i)
    theta_elbow    = -2*np.sin(np.pi/17 * i)
    
    currentPosX, currentPosY, currentPosZ = getCurrentPos(theta_base, theta_shoulder, theta_elbow)
    lines.set_data_3d(currentPosX, currentPosY, currentPosZ)
    balls._offsets3d = (currentPosX, currentPosY, currentPosZ)
    return (lines, balls,)


initPosX, initPosY, initPosZ = getCurrentPos(0, np.pi/2, 0)

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.set_xlim3d(-0.5, 0.5)
ax.set_ylim3d(-0.5, 0.5)
ax.set_zlim3d(0, 1)

lines = ax.plot3D(initPosX, initPosY, initPosZ, 'black')[0]
balls = ax.scatter3D(initPosX, initPosY, initPosZ, c='r');

plt.close()

# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, rotate_base, frames=35, interval=100, blit=True)

HTML(anim.to_html5_video())

RuntimeError: Requested MovieWriter (ffmpeg) not available

## Integrating it into the GUI
What currently exists is:

In [ ]:
# Send message to recived new information
s.send(b"")

# Other information
s.recv(588)

# Position
X  = unpack('!d', bytes.fromhex(str(binascii.hexlify(s.recv(8)).decode("utf-8"))))[0]
Y  = unpack('!d', bytes.fromhex(str(binascii.hexlify(s.recv(8)).decode("utf-8"))))[0]
Z  = unpack('!d', bytes.fromhex(str(binascii.hexlify(s.recv(8)).decode("utf-8"))))[0]
RX = unpack('!d', bytes.fromhex(str(binascii.hexlify(s.recv(8)).decode("utf-8"))))[0]
RY = unpack('!d', bytes.fromhex(str(binascii.hexlify(s.recv(8)).decode("utf-8"))))[0]
RZ = unpack('!d', bytes.fromhex(str(binascii.hexlify(s.recv(8)).decode("utf-8"))))[0]

What we need to change is this: 

In [ ]:
# Send message to recived new information
s.send(b"")

# Receive angle information:
s.recv(252)

BaseAngle        = unpack('!d', bytes.fromhex(str(binascii.hexlify(s.recv(8)).decode("utf-8"))))[0]
ShoulderAngle    = unpack('!d', bytes.fromhex(str(binascii.hexlify(s.recv(8)).decode("utf-8"))))[0]
ElbowAngle       = unpack('!d', bytes.fromhex(str(binascii.hexlify(s.recv(8)).decode("utf-8"))))[0]
ElbowEndAngle    = ElbowAngle
Wrist1Angle      = unpack('!d', bytes.fromhex(str(binascii.hexlify(s.recv(8)).decode("utf-8"))))[0]
Wrist2Angle      = unpack('!d', bytes.fromhex(str(binascii.hexlify(s.recv(8)).decode("utf-8"))))[0]
Wrist3Angle      = unpack('!d', bytes.fromhex(str(binascii.hexlify(s.recv(8)).decode("utf-8"))))[0]

# Other information
s.recv(588)

# Position
X  = unpack('!d', bytes.fromhex(str(binascii.hexlify(s.recv(8)).decode("utf-8"))))[0]
Y  = unpack('!d', bytes.fromhex(str(binascii.hexlify(s.recv(8)).decode("utf-8"))))[0]
Z  = unpack('!d', bytes.fromhex(str(binascii.hexlify(s.recv(8)).decode("utf-8"))))[0]
RX = unpack('!d', bytes.fromhex(str(binascii.hexlify(s.recv(8)).decode("utf-8"))))[0]
RY = unpack('!d', bytes.fromhex(str(binascii.hexlify(s.recv(8)).decode("utf-8"))))[0]
RZ = unpack('!d', bytes.fromhex(str(binascii.hexlify(s.recv(8)).decode("utf-8"))))[0]